# Enable Agent Tutorial Part 4: FDS RAG 챗봇

## 개요

FDS Enable Agent와 Context Builder를 통합하여 대화형 사기 탐지 챗봇을 구현한다.

## 학습 내용

- RAG 기반 대화 시스템 구현
- 실시간 거래 분석
- 자동 분석 보고서 생성

---
## 1. 라이브러리 및 클래스 정의

In [1]:
import os
import json
import yaml
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from typing import Dict, Any, List

import torch
import torch.nn as nn

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
print("라이브러리 임포트 완료")

라이브러리 임포트 완료


In [2]:
class FraudAutoencoder(nn.Module):
    def __init__(self, input_dim=10, latent_dim=4):
        super(FraudAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16), nn.ReLU(),
            nn.Linear(16, 8), nn.ReLU(),
            nn.Linear(8, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 8), nn.ReLU(),
            nn.Linear(8, 16), nn.ReLU(),
            nn.Linear(16, input_dim)
        )
    
    def forward(self, x):
        return self.decoder(self.encoder(x))
    
    def get_reconstruction_error(self, x):
        with torch.no_grad():
            return torch.mean((x - self.forward(x)) ** 2, dim=1)


class FDSEnableAgent:
    def __init__(self, skill_path: str):
        with open(skill_path, 'r', encoding='utf-8') as f:
            self.skill = yaml.safe_load(f)
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.iso_forest = joblib.load(self.skill['model_info']['isolation_forest_path'])
        
        checkpoint = torch.load(self.skill['model_info']['autoencoder_path'], map_location=self.device, weights_only=True)
        self.autoencoder = FraudAutoencoder(checkpoint['input_dim'], checkpoint['latent_dim']).to(self.device)
        self.autoencoder.load_state_dict(checkpoint['model_state_dict'])
        self.autoencoder.eval()
        self.ae_threshold = checkpoint['threshold']
        
        self.scaler = joblib.load(self.skill['model_info']['scaler_path'])
        self.label_encoders = joblib.load(self.skill['model_info']['label_encoders_path'])
        with open(self.skill['model_info']['metadata_path'], 'r', encoding='utf-8') as f:
            self.metadata = json.load(f)
        self.client = OpenAI()
        print(f"FDS Enable Agent 초기화 완료")
    
    def analyze(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        signup = pd.to_datetime(input_data['signup_time'])
        purchase = pd.to_datetime(input_data['purchase_time'])
        
        derived = {
            'time_diff_hours': (purchase - signup).total_seconds() / 3600,
            'signup_hour': signup.hour, 'purchase_hour': purchase.hour,
            'is_weekend': 1 if purchase.dayofweek in [5, 6] else 0,
            'is_night': 1 if 0 <= purchase.hour < 6 else 0
        }
        
        features = np.array([[
            input_data['purchase_value'], input_data['age'], derived['time_diff_hours'],
            derived['signup_hour'], derived['purchase_hour'], derived['is_weekend'], derived['is_night'],
            self.label_encoders['source'].transform([input_data['source']])[0],
            self.label_encoders['browser'].transform([input_data['browser']])[0],
            self.label_encoders['sex'].transform([input_data['sex']])[0]
        ]])
        
        features_scaled = self.scaler.transform(features)
        iso_score = -self.iso_forest.score_samples(features_scaled)[0]
        ae_score = self.autoencoder.get_reconstruction_error(
            torch.FloatTensor(features_scaled).to(self.device)
        ).cpu().numpy()[0]
        
        iso_norm = 1 / (1 + np.exp(-2 * (iso_score - 0.5)))
        ae_norm = 1 / (1 + np.exp(-2 * (ae_score / self.ae_threshold - 1)))
        anomaly_score = (iso_norm + ae_norm) / 2
        
        is_fraud = anomaly_score >= self.metadata['ensemble_threshold']
        if anomaly_score >= 0.85: risk_level = 'CRITICAL'
        elif anomaly_score >= 0.7: risk_level = 'HIGH'
        elif anomaly_score >= 0.5: risk_level = 'MEDIUM'
        else: risk_level = 'LOW'
        
        factors = []
        if derived['time_diff_hours'] < 1: factors.append("매우 빠른 구매")
        if derived['is_night']: factors.append("심야 거래")
        if input_data['purchase_value'] > 100: factors.append("고액 거래")
        if input_data['source'] == 'Direct': factors.append("직접 유입")
        
        return {
            "is_fraud": bool(is_fraud), "risk_level": risk_level,
            "anomaly_score": float(anomaly_score),
            "risk_factors": factors, "input_data": input_data,
            "derived_features": derived, "timestamp": datetime.now().isoformat()
        }
    
    def generate_tool_definition(self) -> Dict[str, Any]:
        return {
            "type": "function",
            "function": {
                "name": "analyze_transaction",
                "description": "거래 데이터를 분석하여 사기 가능성을 탐지한다",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "purchase_value": {"type": "number"},
                        "age": {"type": "integer"},
                        "source": {"type": "string", "enum": ["SEO", "Ads", "Direct"]},
                        "browser": {"type": "string", "enum": ["Chrome", "Safari", "FireFox", "IE", "Opera"]},
                        "sex": {"type": "string", "enum": ["M", "F"]},
                        "signup_time": {"type": "string"},
                        "purchase_time": {"type": "string"}
                    },
                    "required": ["purchase_value", "age", "source", "browser", "sex", "signup_time", "purchase_time"]
                }
            }
        }


class FDSContextBuilder:
    def __init__(self, context_dir: str = 'context_store'):
        self.context_dir = Path(context_dir)
        self.context_dir.mkdir(exist_ok=True)
        self.log_file = self.context_dir / 'analysis_logs.json'
        self.summary_file = self.context_dir / 'fds_summary.json'
        self.knowledge_base_file = self.context_dir / 'fds_knowledge_base.txt'
        self.logs = self._load_logs()
        print(f"Context Builder 초기화 완료 (로그: {len(self.logs)}개)")
    
    def _load_logs(self):
        if self.log_file.exists():
            with open(self.log_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return []
    
    def add_analysis(self, result: Dict[str, Any]):
        self.logs.append(result)
        with open(self.log_file, 'w', encoding='utf-8') as f:
            json.dump(self.logs, f, indent=2, ensure_ascii=False, default=str)
        self._update_files()
    
    def _update_files(self):
        if not self.logs: return
        total = len(self.logs)
        fraud_count = sum(1 for l in self.logs if l['is_fraud'])
        
        summary = {
            "total_analyses": total,
            "fraud_detected": fraud_count,
            "fraud_rate": fraud_count / total,
            "last_updated": datetime.now().isoformat()
        }
        with open(self.summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2)
        
        kb = f"""# FDS 지식 베이스\n\n총 분석: {total}건\n사기 탐지: {fraud_count}건 ({fraud_count/total:.1%})\n"""
        with open(self.knowledge_base_file, 'w', encoding='utf-8') as f:
            f.write(kb)
    
    def get_knowledge_base_content(self) -> str:
        if self.knowledge_base_file.exists():
            with open(self.knowledge_base_file, 'r', encoding='utf-8') as f:
                return f.read()
        return ""
    
    def get_summary(self) -> Dict:
        if self.summary_file.exists():
            with open(self.summary_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}

print("모든 클래스 정의 완료")

모든 클래스 정의 완료


---
## 2. FDSRAGChatbot 클래스 구현

In [3]:
class FDSRAGChatbot:
    """FDS 지식 베이스를 활용하는 RAG 챗봇"""
    
    def __init__(self, agent: FDSEnableAgent, context_builder: FDSContextBuilder):
        self.agent = agent
        self.context_builder = context_builder
        self.client = OpenAI()
        self.conversation_history = []
        print("FDS RAG 챗봇 초기화 완료")
    
    def _get_system_prompt(self) -> str:
        knowledge = self.context_builder.get_knowledge_base_content()
        summary = self.context_builder.get_summary()
        
        return f"""
        당신은 사기 탐지 시스템(FDS) AI 어시스턴트다.
        
        ## 역할
        1. 거래 정보를 받아 사기 가능성을 분석한다
        2. 과거 분석 기록을 바탕으로 통계와 인사이트를 제공한다
        3. 분석 결과를 친절하게 설명한다
        
        ## 현재 통계
        {json.dumps(summary, indent=2, ensure_ascii=False) if summary else '데이터 없음'}
        
        ## 지식 베이스
        {knowledge if knowledge else '데이터 없음'}
        
        ## 응답 규칙
        - 문장은 ~다로 끝낸다
        - 위험도에 따라 적절한 조치를 권장한다
        - 거래 분석 요청 시 analyze_transaction 함수를 호출한다
        """.strip()
    
    def chat(self, user_message: str) -> str:
        self.conversation_history.append({"role": "user", "content": user_message})
        
        messages = [{"role": "system", "content": self._get_system_prompt()}] + self.conversation_history
        tools = [self.agent.generate_tool_definition()]
        
        response = self.client.chat.completions.create(
            model="gpt-4o", messages=messages, tools=tools, tool_choice="auto"
        )
        
        msg = response.choices[0].message
        
        if msg.tool_calls:
            for tc in msg.tool_calls:
                args = json.loads(tc.function.arguments)
                result = self.agent.analyze(args)
                self.context_builder.add_analysis(result)
                
                self.conversation_history.append({"role": "assistant", "content": None, "tool_calls": [tc]})
                self.conversation_history.append({
                    "role": "tool", "tool_call_id": tc.id, "name": "analyze_transaction",
                    "content": json.dumps(result, ensure_ascii=False, default=str)
                })
            
            messages = [{"role": "system", "content": self._get_system_prompt()}] + self.conversation_history
            final = self.client.chat.completions.create(model="gpt-4o", messages=messages)
            assistant_msg = final.choices[0].message.content
        else:
            assistant_msg = msg.content
        
        self.conversation_history.append({"role": "assistant", "content": assistant_msg})
        return assistant_msg
    
    def generate_report(self) -> str:
        summary = self.context_builder.get_summary()
        
        prompt = f"""
        다음 FDS 분석 데이터를 바탕으로 보고서를 작성해달라:
        
        {json.dumps(summary, indent=2, ensure_ascii=False)}
        
        보고서 내용:
        1. 전체 분석 현황
        2. 사기 탐지 패턴
        3. 권장 조치
        4. 개선 방향
        
        문장은 ~다로 끝낸다.
        """
        
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1500
        )
        return response.choices[0].message.content
    
    def reset(self):
        self.conversation_history = []
        print("대화 초기화 완료")

print("FDSRAGChatbot 클래스 정의 완료")

FDSRAGChatbot 클래스 정의 완료


---
## 3. 챗봇 초기화 및 대화 테스트

In [4]:
agent = FDSEnableAgent('skills/fds_agent_skill.yaml')
context_builder = FDSContextBuilder()
chatbot = FDSRAGChatbot(agent, context_builder)

FDS Enable Agent 초기화 완료
Context Builder 초기화 완료 (로그: 8개)
FDS RAG 챗봇 초기화 완료


In [5]:
def demo_chat(msg: str):
    print("="*60)
    print(f"사용자: {msg}")
    print("-"*60)
    response = chatbot.chat(msg)
    print(f"챗봇: {response}")
    print("="*60 + "\n")

In [6]:
demo_chat("안녕하세요! FDS 시스템에 대해 알려주세요.")

사용자: 안녕하세요! FDS 시스템에 대해 알려주세요.
------------------------------------------------------------
챗봇: 안녕하세요! FDS (Fraud Detection System) 시스템은 거래 정보를 분석하여 사기의 가능성을 탐지하는 시스템이에요. 이 시스템은 특히 비정상적이거나 의심스러운 거래 패턴을 식별하여 사기로부터 보호하는 데 도움을 줍니다. FDS는 일반적으로 사용자가 거래를 시작한 후의 다양한 요소를 분석하여 각 거래의 위험 수준을 평가합니다. 이를 통해 금융 기관이나 전자 상거래 플랫폼은 사기성 거래를 신속하게 차단하고 고객의 자산을 보호할 수 있습니다.

또한, FDS는 평균 거래 금액, 사기 탐지율, 평균 이상치 점수, 그리고 거래 간 시간 차이 등의 통계를 바탕으로 사기 가능성을 평가합니다. 위험도 분포나 패턴 분석을 통해 더욱 정밀한 탐지가 가능하도록 지원합니다. 문의사항이 있으면 언제든지 말씀해 주세요!



In [7]:
demo_chat("""
거래 분석 부탁드립니다.
35세 남성이 SEO로 유입되어 Chrome에서 결제했습니다.
1월 15일 10시 30분에 가입하고, 2월 20일 14시 25분에 45달러를 구매했습니다.
""")

사용자: 
거래 분석 부탁드립니다.
35세 남성이 SEO로 유입되어 Chrome에서 결제했습니다.
1월 15일 10시 30분에 가입하고, 2월 20일 14시 25분에 45달러를 구매했습니다.

------------------------------------------------------------
챗봇: 거래 분석 결과, 이 거래는 사기 가능성이 낮은 것으로 평가되었습니다. 위험 수준은 "낮음"이며, 이 거래는 주말이 아닌 평일 낮 시간에 이루어졌기 때문에 추가적인 위험 요소는 발견되지 않았습니다. 이상치 점수는 약 0.34로 낮아, 이 거래는 비교적 정상적이라 판단됩니다.

따라서 현재로서는 특별한 조치가 필요하지 않으며, 고객의 구매 활동을 지속적으로 모니터링하는 것이 좋습니다. 추가적인 문의사항이 있으시면 언제든지 말씀해 주세요!



In [8]:
demo_chat("""
긴급 확인이 필요합니다!
19세 남성이 Direct로 들어와서 Opera 브라우저로 결제했어요.
오늘 새벽 2시 30분에 가입하고 2시 45분에 150달러를 결제했습니다.
사기일까요?
""")

사용자: 
긴급 확인이 필요합니다!
19세 남성이 Direct로 들어와서 Opera 브라우저로 결제했어요.
오늘 새벽 2시 30분에 가입하고 2시 45분에 150달러를 결제했습니다.
사기일까요?

------------------------------------------------------------
챗봇: 이 거래는 사기일 가능성이 높습니다. 위험 수준은 "높음"이며, 주요 리스크 요인은 다음과 같습니다:

1. **매우 빠른 구매**: 가입한 후 매우 짧은 시간 안에 구매가 발생했습니다.
2. **심야 거래**: 새벽 2시대에 발생한 거래로, 비정상적인 시간대입니다.
3. **고액 거래**: 150달러로 금액이 비교적 크며, 일반적인 구매 패턴과는 다릅니다.
4. **직접 유입**: 직접 유입은 비정상적일 수 있습니다.

이상의 이유로, 해당 거래는 사기로부터 보호하기 위해 긴급한 조치가 필요합니다. 즉시 계정을 잠그고, 고객에게 연락하여 거래에 대해 확인하는 것을 권장합니다. 추가적인 도움이 필요하시면 언제든지 말씀해 주세요!



In [9]:
demo_chat("지금까지 분석 통계를 알려주세요.")

사용자: 지금까지 분석 통계를 알려주세요.
------------------------------------------------------------
챗봇: 현재까지 총 11건의 거래 분석이 진행되었으며, 그 중 9건에서 사기가 탐지되었습니다. 사기 탐지율은 약 81.8%입니다. 최근 업데이트는 2026년 1월 9일에 이루어졌습니다.

사기 탐지 비율이 상당히 높기 때문에, 의심스러운 거래에 대한 주의가 필요합니다. 추가적인 통계나 도움이 필요하시면 언제든지 말씀해 주세요!



---
## 4. 분석 보고서 생성

In [10]:
print("=== FDS 분석 보고서 ===")
report = chatbot.generate_report()
print(report)

=== FDS 분석 보고서 ===
**FDS 분석 보고서**

2026년 1월 9일 자로 마지막 업데이트된 FDS(Fraud Detection System) 분석 데이터를 바탕으로 보고서를 작성하였다. 본 보고서는 전체 분석 현황, 사기 탐지 패턴, 권장 조치 및 개선 방향에 대해 포괄적으로 다루고자 한다.

1. **전체 분석 현황**  
   총 10건의 분석이 수행되었으며, 이 중 8건에서 사기가 탐지되었음을 확인할 수 있다. 이에 따른 사기 탐지율은 80%로 나타났으며, 이는 상당히 높은 탐지율을 나타내는 수치이다. 이러한 결과는 FDS의 효과적인 작동 및 높은 신뢰성을 의미한다.

2. **사기 탐지 패턴**  
   검출된 사기 사례들 중 자주 나타나는 패턴은 임의의 고액 거래 시도, 비정상적으로 빈번한 로그인 시도, 정상적인 거래에서 벗어난 거래 유형 등이 포함되어 있다. 이러한 패턴을 기반으로 사기 행위자를 보다 신속하게 파악하고 차단할 수 있다.

3. **권장 조치**  
   - 실시간 모니터링: 고액의 거래가 발생하거나 비정상적인 행동이 포착되는 즉시 알림 시스템을 통해 사전 대처할 수 있도록 실시간 모니터링 강화가 필요하다.
   - 사용자 인증 강화: 2단계 인증 등 추가적인 사용자 인증 과정을 도입하여 제조 기반의 사기 시도를 최소화해야 한다.
   - FDS 알고리즘 개선: 사기 탐지율은 높은 편이나 새로운 유형의 사기를 방지하기 위해 지속적인 알고리즘 개선과 업데이트가 요구된다.

4. **개선 방향**  
   FDS의 탐지 정확도를 향상시키기 위해 기계 학습 모델을 활용한 새로운 패턴 학습이 필요하다. 빅데이터와 인공지능을 활용하여 사기 시도의 실시간 분석 및 예측 정확성을 높이는 방향으로 발전시키는 것이 요구된다. 또한, 탐지된 사기 패턴 데이터베이스를 강화하여 불규칙한 거래를 데이터에 더 빠르고 정확하게 반영할 수 있도록 하는 것이 중요하다.

본 보고서를 통해 FDS의 현재 운용 상황과 함께 추가적인 조치 및 개선 방향을

In [11]:
report_path = Path('context_store/fds_report.md')
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report)
print(f"보고서 저장: {report_path}")

보고서 저장: context_store/fds_report.md
